In [1]:
%pip install -q accelerate peft bitsandbytes transformers trl tensorboard huggingface_hub[cli] xformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.13.1 requires torch==1.13.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.14.1 requires torch==1.13.1, but you have torch 2.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
%%capture
%pip install sentencepiece

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
tokenizer, model = None, None

In [1]:
import os
import torch
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    LlamaForCausalLM,
    AutoTokenizer,
    LlamaTokenizer,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig
from trl import SFTTrainer
import transformers

2023-10-12 12:21:32.547656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 12:21:32.729006: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-12 12:21:33.272765: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-12 12:21:33.272826: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /home/user/anaconda3/lib/libcudart.so
CUDA SETUP: Loading binary /home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/user/anaconda3/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/user/anaconda3/lib/libcudart.so'), PosixPath('/home/user/anaconda3/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
/home/user/anacond

In [2]:
SEED = 42
ITERS = 500
MIN_TOKENS = 180
MAX_TOKENS = 200
MODEL_NAME = "meta-llama/Llama-2-7b-hf"
DATASET_PATH = f"bias_llm/data/llama2-7b-200-google-scholar"
TOPICS = [
    "Indian climate",
    "Indian defense",
    "Indian economy",
    "Indian infrastructure"
]


In [3]:
def get_transformer() -> tuple:
    tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

    model = LlamaForCausalLM.from_pretrained(
      MODEL_NAME,
      device_map={"": 0},
    )

    return tokenizer, model

In [4]:
def get_encoding(tokenizer, prompt: str) -> dict:
    return tokenizer(prompt, return_tensors='pt')

def get_text(tokenizer, embedding: dict) -> str:
    return tokenizer.batch_decode(
    embedding,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
    )[0]

In [5]:
import nltk

def generate_text(tokenizer, model, prompt) -> None:
    create_directory(prompt)
    create_csv_file(prompt)
    file_prompt = prompt.replace(' ', '-')
    
    with open(f'/home/user/Desktop/Sudipto/bias_llm/{file_prompt}-prompts.txt', 'r') as pf:
        for i, title in enumerate(pf.readlines()):
            if len(nltk.word_tokenize(title)) < 2:
                continue
            if i > 500:
                break

            title = title.strip()
            input_embedding = get_encoding(tokenizer, title)

            MAX_LEN = random.randint(MIN_TOKENS, MAX_TOKENS)
            print(f'Generating text for "{title}"... ({i + 1})')

            output_embedding = model.generate(
              input_embedding.input_ids.to('cuda'),
              max_new_tokens=MAX_LEN,
              no_repeat_ngram_size = 2,
              do_sample=True,
              top_k=40,
              top_p=0.9,
              temperature=0.6, # randomly chosen
              eos_token_id=tokenizer.eos_token_id
            )

            generated_text = get_text(tokenizer, output_embedding)
            append_to_csv_file(prompt, title, generated_text, i, MAX_LEN)

def create_directory(prompt: str) -> None:
    if not os.path.exists(f'{DATASET_PATH}/{prompt}'):
        os.makedirs(f'{DATASET_PATH}/{prompt}')

    print(f'Created directory for "{prompt}" at {DATASET_PATH}/{prompt}...')

def create_csv_file(prompt: str) -> None:
    fields = ['Category', 'Prompt', 'Generated Text', 'i', 'MAX_LEN']

    with open(f'{DATASET_PATH}/{prompt}/data.csv', 'w') as f:
        writer = csv.DictWriter(f, fieldnames=fields, lineterminator='\n')
        writer.writeheader()

    print(f'Created CSV file for "{prompt}" at {DATASET_PATH}/{prompt}/data.csv...')

def append_to_csv_file(prompt: str, title, generated_text: str, i: int, MAX_LEN: int) -> None:
    with open(f'{DATASET_PATH}/{prompt}/data.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([prompt, title, generated_text, i, MAX_LEN])

def main() -> None:
    threads = []

#     if tokenizer is None and model is None:
    tokenizer, model = get_transformer()

    for topic in TOPICS:
        thread = Thread(target=generate_text, args=(tokenizer, model, topic,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

In [6]:
from threading import Thread
from time import perf_counter

import os
import csv
import random
import torch

torch.manual_seed(SEED)

start_time = perf_counter()
print("Starting...")
main()
print("Done!")
end_time = perf_counter()

print(f"Time Taken: {end_time - start_time:0.4f} second(s) elapsed")

Starting...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Created directory for "Indian climate" at bias_llm/data/llama2-7b-200-google-scholar/Indian climate...
Created directory for "Indian defense" at bias_llm/data/llama2-7b-200-google-scholar/Indian defense...
Created CSV file for "Indian climate" at bias_llm/data/llama2-7b-200-google-scholar/Indian climate/data.csv...
Created directory for "Indian economy" at bias_llm/data/llama2-7b-200-google-scholar/Indian economy...
Created directory for "Indian infrastructure" at bias_llm/data/llama2-7b-200-google-scholar/Indian infrastructure...
Created CSV file for "Indian defense" at bias_llm/data/llama2-7b-200-google-scholar/Indian defense/data.csv...
Created CSV file for "Indian economy" at bias_llm/data/llama2-7b-200-google-scholar/Indian economy/data.csv...
Created CSV file for "Indian infrastructure" at bias_llm/data/llama2-7b-200-google-scholar/Indian infrastructure/data.csv...
Generating text for "Infrastructure in India"... (1)
Generating text for "Impact of climate change on Indian agricul

Generating text for "Physical and cooking characteristics of two cowpea cultivars grown in temperate Indian climate"... (18)
Generating text for "Infrastructure in India: The economics of transition from public to private provision"... (20)
Generating text for "us-india defence ties: the limits to interoperability"... (19)
Generating text for "the indian economy, 1947-92. volume 2: population, poverty and employment."... (20)
Generating text for "Syndication of VCPE investments: Evidence from the Indian infrastructure sector"... (21)
Generating text for "Life cycle cost analysis of HPVT air collector under different Indian climatic conditions"... (19)
Generating text for "make in india in indian defence: a prospect of development for micro, small & medium enterprises"... (20)
Generating text for "social accounting matrix to study the socio-economic linkages of tourism sector–a case study of india"... (21)
Generating text for "Infrastructure development in India"... (22)
Generating text

Generating text for "Impact of climate change on Indian agriculture"... (36)
Generating text for "did india's economic reforms improve efficiency and productivity? a nonparametric analysis of the initial evidence from manufacturing"... (39)
Generating text for "ballistic missile defence: implications for india-pakistan strategic environment"... (38)
Generating text for "Chinese roads in India: The effect of transport infrastructure on economic development"... (39)
Generating text for "Environmental justice discourses in Indian climate politics"... (37)
Generating text for "the political economy of technological innovation in indian agriculture"... (40)
Generating text for "Structure and mechanisms of South Indian Ocean climate variability"... (38)
Generating text for "indian defence program 1964-78"... (39)
Generating text for "Governance in India: Infrastructure"... (40)
Generating text for "Infrastructure in India: challenges and the way ahead"... (41)
Generating text for "what deter

...

Generating text for "Carbon emissions, climate change, and impacts in India's cities"... (411)
Generating text for "indian agriculture-status, importance and role in indian economy"... (409)
Generating text for "lifecycle management using risk estimation and contingency model for indian defence sector"... (415)
Generating text for "Interdecadal variation of Indian Ocean basin mode and the impact on Asian summer climate"... (404)
Generating text for "Infrastructure: its development and impact on agriculture in north-east india"... (412)
Generating text for "uncertain success: the political economy of indian economic reform"... (410)
Generating text for "india-us defence trade continuity under trump"... (416)
Generating text for "Climatic response of the Indian subcontinent to doubled CO2 concentrations"... (405)
Generating text for "Co-production or contested production? Complex arrangements of actors, infrastructure, and practices in everyday water provisioning in a small town in India

Generating text for "Simulated projections for summer monsoon climate over India by a high-resolution regional climate model (PRECIS)"... (421)
Generating text for "Creative financing of urban infrastructure in India through market-based financing and public-private partnership options"... (429)
Generating text for "india in australia's 2016 defence white paper"... (433)
Generating text for "the performance of the indian economy in the 1990s"... (429)
Generating text for "Relative vulnerability of Indian coastal districts to sea-level rise and climate extremes"... (422)
Generating text for "transformation of indian economy into knowledge economy–prospects and challenges"... (430)
Generating text for "Infrastructure financing at crossroads: the case of Infrastructure Leasing and Financial Services Ltd.(India)"... (430)
Generating text for "thomas," the defence of india: a budgetary perspective of strategy and politics"(book review)"... (434)
Generating text for "india's defence spending

Generating text for "Climate change and its impact on tropical montane ecosystems in southern India"... (437)
Generating text for "private defence under indian criminal law"... (451)
Generating text for "Potential of Land and Land-based Instruments for Infrastructure Development in Urban Areas"... (447)
Generating text for "politics of economic growth in india, 1980-2005: part i: the 1980s"... (449)
Generating text for "Regional climate model applications on sub‐regional scales over the Indian monsoon region: The role of domain size on downscaling uncertainty"... (438)
Generating text for "Quality, accreditation and global university ranking: Issues before Indian higher education"... (448)
Generating text for "make in india and defence manufacturing sector"... (452)
Generating text for "the software industry and india's economic development"... (450)
Generating text for "Measuring safety culture on ships using safety climate: a study among Indian officers"... (439)
Generating text for 

Generating text for "novel corona virus pandemic-impact on indian economy, e-commerce, education and employment"... (466)
Generating text for "Climate risk, vulnerability and resilience: Supporting livelihood of smallholders in semiarid India"... (456)
Generating text for "where phalcons dare: india-israel defence relations"... (470)
Generating text for "Globalisation and the changing face of port infrastructure: The Indian perspective"... (467)
Generating text for "why worry about inequality in the booming indian economy?"... (467)
Generating text for "Indian summer monsoon rainfall processes in climate change scenarios"... (457)
Generating text for "ASEAN-India Connectivity: A Regional Framework and Key Infrastructure Projects"... (468)
Generating text for "indian defence review jul-sep 2015 (vol 30.3)"... (471)
Generating text for "indian defence review vol 31.2 (apr-jun 2016): 31.2"... (472)
Generating text for "the social & economical impact of gst on indian economy"... (468)
Gene

Generating text for "Climate change and sustainable livelihood programme: a case study from Indian Sundarbans"... (476)
Generating text for "Village transportation infrastructure and women's non-agricultural employment in India: The conditioning role of community gender context"... (486)
Generating text for "indian military defence"... (490)
Generating text for "Progress towards achieving the challenge of Indian summer monsoon climate simulation in a coupled ocean‐atmosphere model"... (477)
Generating text for "National development banks and sustainable infrastructure in South Asia"... (487)
Generating text for "daily newspapers' views on defence science & technology in india"... (491)
Generating text for "Infrastructure and regional development in India"... (488)
Generating text for "Projection of Indian summer monsoon climate in 2041–2060 by multiregional and global climate models"... (478)
Generating text for "China's Belt and Road Initiative and India's options: Competitive coopera

In [ ]:
# ITERS = 2000

# from threading import Thread
# from time import perf_counter

# import os
# import csv
# import random
# import torch

# torch.manual_seed(SEED)

# start_time = perf_counter()
# print("Starting...")
# main()
# print("Done!")
# end_time = perf_counter()

# print(f"Time Taken: {end_time - start_time:0.4f} second(s) elapsed")

In [17]:
import torch

In [18]:
[torch.cuda.device(i) for i in range(torch.cuda.device_count())]

In [8]:
import pandas as pd

In [9]:
dfs = {}

for topic in TOPICS:
  df = pd.read_csv(f"{DATASET_PATH}/{topic}/data.csv")
  dfs[topic] = df

In [10]:
dfs['Indian economy']

,Prompt,Generated Text,i,MAX_LEN
0,Indian economy,Indian economy likely to grow 8-8.5% on strong...,0,47
1,Indian economy,Indian economy is in doldrums due to weak GDP ...,1,57
2,Indian economy,"Indian economy will soon bounce back, says Jai...",2,46
3,Indian economy,Indian economy slows down as demand weakens\nS...,3,58
4,Indian economy,Indian economy's potential to create more than...,4,54
...,...,...,...,...
1995,Indian economy,Indian economy grows 9.5% in Q2\nNew Delhi (AN...,1995,57
1996,Indian economy,Indian economy likely to get ‘strong fillip’ a...,1996,52
1997,Indian economy,Indian economy to grow at fastest rate among e...,1997,54
1998,Indian economy,Indian economy likely to grow by 5-5.5 per cen...,1998,51


In [11]:
dfs['Indian climate']

,Prompt,Generated Text,i,MAX_LEN
0,Indian climate,Indian climate is tropical and monsoon with a ...,0,49
1,Indian climate,Indian climate - A great place for wine? - Par...,1,57
2,Indian climate,Indian climate experts to brief Bangladeshi cl...,2,50
3,Indian climate,Indian climate has four main seasons i.e. summ...,3,52
4,Indian climate,Indian climate\nThe climate in India is mainly...,4,55
...,...,...,...,...
1995,Indian climate,"Indian climate, economy to suffer in absence o...",1995,48
1996,Indian climate,Indian climate.\n(Indian climate is characteri...,1996,50
1997,Indian climate,Indian climate change\nI. K. MANDALA\nTHE firs...,1997,46
1998,Indian climate,Indian climate scientist: Rice has adapted to ...,1998,49


In [12]:
dfs['Indian defense']

,Prompt,Generated Text,i,MAX_LEN
0,Indian defense,Indian defense contractor to display cutting-e...,0,47
1,Indian defense,Indian defense secretary arrives for Afghan se...,1,56
2,Indian defense,"Indian defense deals, India’s foreign policy, ...",2,51
3,Indian defense,Indian defense contractor Hindustan Aeronautic...,3,60
4,Indian defense,"Indian defense minister meets China’s Jinping,...",4,60
...,...,...,...,...
1995,Indian defense,Indian defense is trying to figure out ways to...,1995,46
1996,Indian defense,"Indian defense, finance ministers urge BRICS t...",1996,49
1997,Indian defense,Indian defense has been in doldrums after defe...,1997,55
1998,Indian defense,Indian defense personnel receive French-made L...,1998,59


In [13]:
dfs['Indian infrastructure']

,Prompt,Generated Text,i,MAX_LEN
0,Indian infrastructure,Indian infrastructure: A tale of two cities\nI...,0,52
1,Indian infrastructure,Indian infrastructure to see a sea change duri...,1,47
2,Indian infrastructure,Indian infrastructure has been improving but w...,2,58
3,Indian infrastructure,Indian infrastructure investment funds and alt...,3,58
4,Indian infrastructure,Indian infrastructure projects are likely to b...,4,58
...,...,...,...,...
1995,Indian infrastructure,Indian infrastructure growth could be a good i...,1995,53
1996,Indian infrastructure,Indian infrastructure sector was at the boom w...,1996,50
1997,Indian infrastructure,Indian infrastructure funds attractive: KKR\nT...,1997,47
1998,Indian infrastructure,Indian infrastructure growth is slow and unsus...,1998,48


In [14]:
df_eco = dfs['Indian economy']
texts = df_eco['Generated Text']

In [15]:
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>
[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 104] Connection reset by peer>


False

In [17]:
for text in texts[8:]:
  print('text:', text)
  sentences = sent_tokenize(text)
  for sentence in sentences:
    print('------')
    print('sentence:', sentence)
    words = word_tokenize(sentence)
    print('pos tags:', nltk.pos_tag(words))
    words = [word.lower() for word in words if word not in stopwords.words('english') and word.isalpha()]
    print('processed:', words)
  break

text: Indian economy not in recession and has room for recovery: RBI Governor
Sugata Ghosh / TNN / Jun 10, 2021,
Mumbai: Reserve Bank of India governor Shaktik
------
sentence: Indian economy not in recession and has room for recovery: RBI Governor
Sugata Ghosh / TNN / Jun 10, 2021,
Mumbai: Reserve Bank of India governor Shaktik
pos tags: [('Indian', 'JJ'), ('economy', 'NN'), ('not', 'RB'), ('in', 'IN'), ('recession', 'NN'), ('and', 'CC'), ('has', 'VBZ'), ('room', 'NN'), ('for', 'IN'), ('recovery', 'NN'), (':', ':'), ('RBI', 'NNP'), ('Governor', 'NNP'), ('Sugata', 'NNP'), ('Ghosh', 'NNP'), ('/', 'NNP'), ('TNN', 'NNP'), ('/', 'NNP'), ('Jun', 'NNP'), ('10', 'CD'), (',', ','), ('2021', 'CD'), (',', ','), ('Mumbai', 'NNP'), (':', ':'), ('Reserve', 'NNP'), ('Bank', 'NNP'), ('of', 'IN'), ('India', 'NNP'), ('governor', 'NN'), ('Shaktik', 'NNP')]
processed: ['indian', 'economy', 'recession', 'room', 'recovery', 'rbi', 'governor', 'sugata', 'ghosh', 'tnn', 'jun', 'mumbai', 'reserve', 'bank', 'i